# Hazelcast Python Client SQL Support Notebook

This notebook shows example usage of Hazelcast Python Client using SQL functions. Hazelcast provides in-depth SQL support for your data kept in Hazelcast clusters. You can create a mapping between a database table and your data resource and execute SQL queries on the map. 


To simplify setup, this tutorial uses Hazelcast Cloud for the cluster service. Hazelcast Cloud offers free registration with 200MB storage area. You can register the Hazelcast Cloud using following link and create a cluster in few minutes: https://cloud.hazelcast.com/. You can follow this tutorial to create a cluster in Cloud. If you have any issue, you can ask your questions in Hazelcast Community Slack channel. 

In [9]:
%pip install ipywidgets

import ipywidgets as widgets

/Users/hazelcast/.zshenv:1: permission denied: /Users/hazelcast/Desktop


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%jupyter` not found.


In [12]:
a = widgets.FloatLogSlider(
    value=10,
    base=10,
    min=-10, # max exponent of base
    max=10, # min exponent of base
    step=0.2, # exponent step
    description='Log Slider'
)
display(a)
a
widgets.Output(a)

FloatLogSlider(value=10.0, description='Log Slider', max=10.0, min=-10.0, step=0.2)

TypeError: Widget.__init__() takes 1 positional argument but 2 were given

![Tux, the Linux mascot](https://media-exp1.licdn.com/dms/image/C561BAQEDzMqseo-qiA/company-background_10000/0/1626235318986?e=2147483647&v=beta&t=9QQp_S3AuK8-1nMqsSTlL0xy2_JpkNQrzFJ7D_SLUYU)


After the creation of cluster, we need to connect it with a client program. You can connect to your cluster in six programming languages including Java, .NET, Python, Go+.  To use Hazelcast, you need to have one of the client in your local computer and compile/run your program with it. Following install command downloands Hazelcast Python Client locally if you do not done it before. 

Also, we will pull over data from an API, and we will send request. So, we need requests module too.

# Install packages

In [1]:
%pip install hazelcast-python-client
%pip install requests

/Users/hazelcast/.zshenv:1: permission denied: /Users/hazelcast/Desktop
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
/Users/hazelcast/.zshenv:1: permission denied: /Users/hazelcast/Desktop
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Imports

In [2]:
import hazelcast
import requests
import logging
from hazelcast.core import HazelcastJsonValue

# Connect To Hazelcast Cloud

In [4]:
logging.basicConfig(level=logging.INFO)

client = hazelcast.HazelcastClient()

print("Connection successfull.")

INFO:hazelcast.lifecycle:HazelcastClient 5.1 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.1 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.1 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=127.0.0.1, port=5701):d479fc9f-4229-4de9-8ac8-9e66755c841e, server version: 5.1.2, local address: Address(host=127.0.0.1, port=49554)
INFO:hazelcast.cluster:

Members [1] {
	Member [127.0.0.1]:5701 - d479fc9f-4229-4de9-8ac8-9e66755c841e
}

INFO:hazelcast.client:Client started


Connection successfull.


# Create Mapping between Map and Table

In [21]:
prices = client.get_map("prices").blocking()

In [22]:
exchange = client.get_map("exchanges").blocking()

In [23]:
def create_mapping(hz_client):
    # see: https://docs.hazelcast.com/hazelcast/5.0/sql/mapping-to-maps#json-objects
    print("Creating mapping for countries...")

    mapping_query = """
        CREATE OR REPLACE MAPPING prices (
            __key VARCHAR, 
            symbol VARCHAR,
            priceChange DOUBLE,
            priceChangePercent DECIMAL,
            weightedAvgPrice DECIMAL,
            prevClosePrice DECIMAL,
            lastPrice DECIMAL,
            lastQty DECIMAL,
            bidPrice DECIMAL,
            bidQty DECIMAL,
            askPrice DECIMAL,
            askQty DECIMAL,
            openPrice DECIMAL,
            highPrice DECIMAL,
            lowPrice DECIMAL,
            volume VARCHAR,
            quoteVolume VARCHAR,
            openTime BIGINT,
            closeTime BIGINT,
            firstId BIGINT,
            lastId BIGINT
        )
        TYPE IMap OPTIONS(
            'keyFormat' = 'varchar',
            'valueFormat' = 'json-flat'
        )
    """
    hz_client.sql.execute(mapping_query).result()
    print("Mapping for countries has been created.")
    print("--------------------")


In [24]:
def create_mapping_a(hz_client):
    # see: https://docs.hazelcast.com/hazelcast/5.0/sql/mapping-to-maps#json-objects
    print("Creating mapping for countries...")

    mapping_query = """
        CREATE OR REPLACE MAPPING exchanges (
            __key VARCHAR, 
            symbol VARCHAR,
            status VARCHAR,
            baseAsset VARCHAR,
            baseAssetPrecision INT,
            quoteAsset VARCHAR,
            quotePrecision INT,
            quoteAssetPrecision INT,
            baseCommissionPrecision INT,
            quoteCommissionPrecision INT,
            icebergAllowed BOOLEAN,
            ocoAllowed BOOLEAN,
            quoteOrderQtyMarketAllowed BOOLEAN,
            allowTrailingStop BOOLEAN,
            cancelReplaceAllowed BOOLEAN,
            isSpotTradingAllowed BOOLEAN,
            isMarginTradingAllowed BOOLEAN
        )
        TYPE IMap OPTIONS(
            'keyFormat' = 'varchar',
            'valueFormat' = 'json-flat'
        )
    """
    hz_client.sql.execute(mapping_query).result()

In [25]:
create_mapping(client)

Creating mapping for countries...
Mapping for countries has been created.
--------------------


In [26]:
create_mapping_a(client)

Creating mapping for countries...


# Load Data from API

In [27]:
def load_rates(prices):
    data = requests.get('https://www.binance.com/api/v3/ticker/24hr').json()
    
    i = 0;
    for f in data:
        prices.put(str(i), HazelcastJsonValue(f))
        i += 1
            
    print("Data has been loaded.")

In [28]:
load_rates(prices)

Data has been loaded.


In [29]:
def load_exchange_info(prices):
    data = requests.get('https://www.binance.com/api/v3/exchangeInfo').json()
  
    i = 0;
    for f in data['symbols']:
        exchange.put(str(i), HazelcastJsonValue(f))
        i += 1
            
    print("Data has been loaded.")

In [30]:
load_exchange_info(prices)

Data has been loaded.


# Query 